In [10]:
import cv2
import numpy as np

from keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError

In [11]:
faces_path = 'haarcascade_frontalface_default.xml'
emotion_path = 'emotion.h5'
age_path = 'age.h5'
skin_path = 'skin.h5'

In [12]:
emotion = load_model(emotion_path)
emotion.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 48, 48, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 48, 48, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 24, 24, 128)    │       409,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 6, 6, 128)      │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 6, 6, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,863,689 (7.11 MB)

 Trainable params: 1,861,255 (7.10 MB)

 Non-trainable params: 2,432 (9.50 KB)

 Optimizer params: 2 (12.00 B)

In [13]:
emotion_map = {
    0: 'Anger',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happy',
    4: 'Sad',
    5: 'Surprise',
    6: 'Neutral'
}

skin_advice_map = {
    "Acne": {
        "Anger": "Avoid touching your face to prevent breakouts.",
        "Disgust": "Use a gentle cleanser to clear clogged pores.",
        "Fear": "Try a soothing gel with aloe vera to calm redness.",
        "Happy": "Keep your routine simple to avoid irritation.",
        "Sad": "Hydrate well and use non-comedogenic products.",
        "Surprise": "Spot treatments can help with sudden breakouts.",
        "Neutral": "Stick to oil-free skincare for balanced skin."
    },
    "Dry": {
        "Anger": "Apply a rich moisturizer to soothe dry patches.",
        "Disgust": "Use a hydrating serum to replenish lost moisture.",
        "Fear": "A humidifier can help keep skin from drying out.",
        "Happy": "Drink plenty of water to keep skin soft and smooth.",
        "Sad": "Try a nourishing face mask to restore hydration.",
        "Surprise": "Use a facial mist to keep your skin refreshed.",
        "Neutral": "A daily moisturizer is key for healthy skin."
    },
    "Normal": {
        "Anger": "Avoid harsh scrubs; stick to calming products.",
        "Disgust": "Try a light moisturizer to refresh your skin.",
        "Fear": "Keep your routine gentle and hydrating.",
        "Happy": "Maintain your healthy glow with hydration!",
        "Sad": "A nourishing mask can lift your skin's mood.",
        "Surprise": "Use gentle cleansers to maintain balance.",
        "Neutral": "Keep using your balanced skincare routine."
    },
    "Oily": {
        "Anger": "Use oil-free products to avoid breakouts.",
        "Disgust": "Clay masks can help control excess oil.",
        "Fear": "Apply a mattifying moisturizer for balance.",
        "Happy": "Keep blotting papers handy for fresh skin!",
        "Sad": "Use a mild cleanser to prevent acne flare-ups.",
        "Surprise": "Hydrating gels can help without excess oil.",
        "Neutral": "A lightweight sunscreen is a great choice."
    }
}


In [14]:
def draw_text_with_bg(image, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.75, text_color=(0, 0, 255), bg_color=(0, 0, 0), thickness=2):
    """
    Draw text with background in OpenCV.
    """
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    x, y = position
    bg_x1, bg_y1 = x - 5, y - text_size[1] - 5  # Background padding
    bg_x2, bg_y2 = x + text_size[0] + 5, y + 5

    # Draw the background rectangle
    cv2.rectangle(image, (bg_x1, bg_y1), (bg_x2, bg_y2), bg_color, -1)

    # Draw the text over the background
    cv2.putText(image, text, (x, y), font, font_scale, text_color, thickness)

def draw_multiline_text_with_bg(img, text, pos, max_width=150, font_scale=0.5, text_color=(0, 0, 0), bg_color=(255, 255, 255), line_spacing=8):
    """
    Draws multi-line text inside a box with a background.
    Automatically wraps text if it exceeds max_width.
    line_spacing: Additional space between lines.
    """
    font = cv2.FONT_HERSHEY_SIMPLEX
    thickness = 1
    base_line_height = int(20 * font_scale)  # Base line height
    line_height = base_line_height + line_spacing  # Increase spacing

    # Word wrapping
    words = text.split()
    lines = []
    current_line = ""

    for word in words:
        test_line = current_line + " " + word if current_line else word
        text_size = cv2.getTextSize(test_line, font, font_scale, thickness)[0]

        if text_size[0] > max_width:
            lines.append(current_line)
            current_line = word
        else:
            current_line = test_line

    if current_line:
        lines.append(current_line)

    # Compute the box dimensions
    text_width = max([cv2.getTextSize(line, font, font_scale, thickness)[0][0] for line in lines])
    text_height = len(lines) * line_height + 10  # Adjusted for spacing

    x, y = pos
    box_tl = (x, y)
    box_br = (x + text_width + 10, y + text_height)

    # Ensure the box doesn't go outside the image boundaries
    frame_height, frame_width, _ = img.shape
    if box_br[0] > frame_width:
        box_tl = (frame_width - text_width - 20, y)
        box_br = (frame_width - 10, y + text_height)

    cv2.rectangle(img, box_tl, box_br, bg_color, -1)  # Draw background box

    # Draw each line of text
    for i, line in enumerate(lines):
        text_pos = (box_tl[0] + 5, box_tl[1] + (i + 1) * line_height)
        cv2.putText(img, line, text_pos, font, font_scale, text_color, thickness, cv2.LINE_AA)



In [15]:
age = load_model(age_path, custom_objects={'mae': MeanAbsoluteError()})
age.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,179 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 2 (12.00 B)

In [16]:
skin = load_model(skin_path)
skin.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 128, 128, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 4,484,262 (17.11 MB)

 Trainable params: 4,483,108 (17.10 MB)

 Non-trainable params: 1,152 (4.50 KB)

 Optimizer params: 2 (12.00 B)

In [17]:
skin_map = {
    0: 'Acne',
    1: 'Dry',
    2: 'Normal',
    3: 'Oily'
}

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # Set width to 1920 pixels
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # Set height to 1080 pixels

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

last_predictions = {}  # Store last known predictions

while True:
    ret, frame = cap.read()
    if not ret or frame is None:  # Ensure frame is valid
        continue

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    faces = sorted(faces, key=lambda x: x[0])

    for i, (x, y, w, h) in enumerate(faces):
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        key = cv2.waitKey(1) & 0xFF  # Ensure key press works correctly

        if key == 32:  # Spacebar for prediction
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = frame[y:y + h, x:x + w]

            try:
                # Emotion prediction
                roi_gray_emotion = cv2.resize(roi_gray, (48, 48))
                roi_gray_emotion = roi_gray_emotion / 255.0
                roi_gray_emotion = roi_gray_emotion.reshape(1, 48, 48, 1)
                pred = emotion.predict(roi_gray_emotion)
                predicted_index = np.argmax(pred)
                predicted_emotion = emotion_map.get(predicted_index, "Unknown")

                # Age prediction
                roi_color_age = cv2.resize(roi_color, (128, 128))
                roi_color_age = roi_color_age / 255.0
                roi_color_age = roi_color_age.reshape(1, 128, 128, 3)
                age_pred = age.predict(roi_color_age)
                predicted_age = int(age_pred[0][0]) if age_pred is not None else 0

                # Skin prediction
                roi_color_skin = cv2.resize(roi_color, (128, 128))
                roi_color_skin = roi_color_skin / 255.0
                roi_color_skin = roi_color_skin.reshape(1, 128, 128, 3)
                skin_pred = skin.predict(roi_color_skin)
                predicted_skin = skin_map.get(np.argmax(skin_pred), "Unknown")

                # Store predictions
                last_predictions[i] = (predicted_emotion, predicted_age, predicted_skin)

            except Exception as e:
                print(f"Prediction error: {e}")
                last_predictions[i] = ("Unknown", 0, "Unknown")

        # Use last stored predictions if available
        predicted_emotion, predicted_age, predicted_skin = last_predictions.get(i, ("Unknown", 0, "Unknown"))

        # Display predictions with background
        draw_text_with_bg(frame, predicted_emotion, (x + 10, y - 10), text_color=(255, 255, 255), bg_color=(0, 0, 0))
        draw_text_with_bg(frame, f"Age: {predicted_age}", (x + 10, y + h + 30), text_color=(255, 255, 255), bg_color=(50, 50, 50))
        draw_text_with_bg(frame, predicted_skin, (x + 10, y + h + 60), text_color=(255, 255, 255), bg_color=(100, 100, 100))

        # Display skin advice for each face
        skin_advice = skin_advice_map.get(predicted_skin, {}).get(predicted_emotion, "")

        if skin_advice:
            advice_x = x + 10
            advice_y = y + h + 90  # Position below the face box
            draw_multiline_text_with_bg(frame, skin_advice, (x + w + 10, y), max_width=150, font_scale=0.5)


    # Display the frame
    cv2.imshow('frame', frame)

    # Move cv2.waitKey() here for better responsiveness
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 490ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━